In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
dfs = []
for i in range(1,5):
    path = 'all data/UNSW-NB15_{}.csv'  # There are 4 input csv files
    dfs.append(pd.read_csv(path.format(i), header = None))
combined_data = pd.concat(dfs).reset_index(drop=True)  # Concat all to a single df

dataset_columns = pd.read_csv('NUSW-NB15_features.csv',encoding='ISO-8859-1')
dataset_columns.info()

combined_data.columns = dataset_columns['Name']
combined_data.info()

C:\Users\user\AppData\Local\Temp\ipykernel_25440\2345154628.py:4: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path.format(i), header = None))
C:\Users\user\AppData\Local\Temp\ipykernel_25440\2345154628.py:4: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path.format(i), header = None))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No.          49 non-null     int64 
 1   Name         49 non-null     object
 2   Type         49 non-null     object
 3   Description  49 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540047 entries, 0 to 2540046
Data columns (total 49 columns):
 #   Column            Dtype  
---  ------            -----  
 0   srcip             object 
 1   sport             object 
 2   dstip             object 
 3   dsport            object 
 4   proto             object 
 5   state             object 
 6   dur               float64
 7   sbytes            int64  
 8   dbytes            int64  
 9   sttl              int64  
 10  dttl              int64  
 11  sloss             int64  
 12  dloss             int64  
 13  service           o

In [3]:
import matplotlib.pyplot as plt

class_counts = combined_data['Label'].value_counts()

classes = class_counts.index
counts = class_counts.values

class_0 = combined_data[combined_data['Label'] == 0]
class_1 = combined_data[combined_data['Label'] == 1]

# 計算新的樣本數量
new_class_0_size = int(len(class_0) * 0.1)
new_class_1_size = int(len(class_1)*0.1)

# 對兩個類別進行下採樣
class_0_under = class_0.sample(new_class_0_size, random_state=42)
class_1_under = class_1.sample(new_class_1_size, random_state=42)

combined_data = pd.concat([class_0_under, class_1_under])

class_counts = combined_data['Label'].value_counts()

classes = class_counts.index
counts = class_counts.values

attack_cat_counts = combined_data['attack_cat'].value_counts()
labels = ['Normal', 'Generic', 'Exploits', 'Fuzzers', 'DoS', 'Reconnaissance', 'Analysis', 'Backdoor', 'Shellcode', 'Worms']


In [4]:
new_class_0_size,new_class_1_size

(221876, 32128)

In [5]:
combined_data['attack_cat'] = combined_data['attack_cat'].fillna(value='normal').apply(lambda x: x.strip().lower())

combined_data['ct_flw_http_mthd'] = combined_data['ct_flw_http_mthd'].fillna(value=0)
combined_data['is_ftp_login'] = combined_data['is_ftp_login'].fillna(value=0)
combined_data['is_ftp_login'] = np.where(combined_data['is_ftp_login']>1, 1, combined_data['is_ftp_login'])
combined_data['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)

combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)
combined_data.drop(columns=['srcip','sport','dstip','dsport','Label'],inplace=True)
combined_data.drop(columns=['sloss','Dload','Spkts','trans_depth','tcprtt','synack','is_ftp_login'],inplace=True)

# Remplacer tous les noms des attaques par "attack", sauf "normal"
combined_data['attack_cat'] = combined_data['attack_cat'].apply(
    lambda x: 'attack' if x.lower() != 'normal' else 'Normal'
)

train, test = train_test_split(combined_data,test_size=0.2,random_state=16)

num_col = train.select_dtypes(include=['number']).columns


In [6]:

#Data Preprocessing¶
scaler = StandardScaler()
scaler = scaler.fit(train[num_col])
train[num_col] = scaler.transform(train[num_col])
test[num_col] = scaler.transform(test[num_col])


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Supposons que cat_col contient les noms des colonnes catégorielles
cat_col = ['proto', 'service', 'state']  # exemple

# Toutes les autres colonnes sont numériques
num_col = ['dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'Sload', 'Dpkts', 
           'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 
           'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 
           'Dintpkt', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 
           'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'dloss',
           'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm','ackdat',
        #    'sloss','Dload','Spkts','trans_depth','tcprtt','synack','is_ftp_login',  #沒刪特徵
           'attack_cat']

# 確保 cat_col + num_col 覆蓋了數據框的所有列
all_columns = cat_col + num_col

# 將 numpy.ndarray 轉換為 pandas DataFrame
train = pd.DataFrame(train, columns=all_columns)
test = pd.DataFrame(test, columns=all_columns)


In [8]:
train

,proto,service,state,dur,sbytes,dbytes,sttl,dttl,Sload,Dpkts,...,ct_srv_src,ct_srv_dst,ct_dst_ltm,dloss,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ackdat,attack_cat
151381,tcp,None,FIN,-0.179189,-0.043013,-0.214546,-0.425611,-0.042251,-0.292037,-0.155597,...,-0.387957,0.001590,-0.543145,-0.185037,-0.353251,-0.428641,-0.418546,-0.518691,-0.109611,Normal
1944180,udp,dns,INT,-0.180841,-0.075806,-0.229321,2.563768,-0.718245,0.121049,-0.358966,...,1.919561,1.943194,1.912526,-0.294592,1.843156,2.053984,1.361704,2.058655,-0.115812,attack
391879,udp,dns,CON,-0.180575,-0.075237,-0.228182,-0.425611,-0.042251,-0.308516,-0.342019,...,-0.572558,-0.645611,-0.297578,-0.294592,0.012817,-0.310421,-0.418546,-0.252069,-0.115812,Normal
2153074,tcp,ssh,FIN,0.568312,0.143962,-0.145920,-0.425611,-0.042251,-0.313432,0.200299,...,-0.757160,-0.738068,-0.174795,0.015815,-0.231228,-0.428641,-0.418546,-0.518691,-0.110703,Normal
2429901,udp,dns,INT,-0.180842,-0.075806,-0.229321,2.563768,-0.718245,0.245276,-0.358966,...,2.011861,2.035651,1.053041,-0.294592,1.233043,1.226442,1.847227,2.147529,-0.115812,attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615488,tcp,None,FIN,-0.173086,-0.013564,0.071122,-0.425611,-0.042251,-0.304942,0.166404,...,0.165847,-0.460696,-0.543145,0.143630,-0.231228,-0.428641,-0.418546,-0.518691,-0.110469,Normal
851784,tcp,None,FIN,0.786707,0.061269,-0.134346,-0.425611,-0.042251,-0.313593,0.708721,...,-0.018754,-0.460696,-0.543145,0.289704,-0.353251,-0.428641,-0.418546,-0.429817,-0.110859,Normal
1554850,tcp,smtp,FIN,-0.042846,0.582893,-0.207684,-0.425611,-0.042251,-0.308672,-0.003070,...,-0.757160,-0.645611,-0.420362,-0.148518,0.256862,-0.428641,-0.418546,-0.340943,-0.110157,Normal
1402766,udp,dns,INT,-0.180843,-0.075806,-0.229321,2.563768,-0.718245,0.990641,-0.358966,...,2.842568,2.867767,1.053041,-0.294592,0.988998,1.226442,1.847227,2.947395,-0.115812,attack


In [9]:
# 創建 ColumnTransformer，將 OneHotEncoder 應用於類別列
ct = ColumnTransformer(
    transformers=[('', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_col)], 
    remainder='passthrough'
)

# 適用於訓練集並轉換
x_train_transformed = ct.fit_transform(train)
# 轉換測試集和驗證集
x_test_transformed = ct.transform(test)

# 獲取編碼後的特徵名稱
new_feature_names = ct.get_feature_names_out()
new_feature_names = [name.replace('remainder', '') for name in new_feature_names]  # 移除前綴
new_feature_names = [name.replace('__', '') for name in new_feature_names]  # 移除前綴
# 創建新的 DataFrame，包含轉換後的數據和新特徵名稱
x_train = pd.DataFrame(x_train_transformed, columns=new_feature_names)
x_test = pd.DataFrame(x_test_transformed, columns=new_feature_names)

In [10]:
replace_dict = {'Normal': 0, 'attack': 1}

# 替換 'attack_cat' 欄位的值
x_train['attack_cat'] = x_train['attack_cat'].replace(replace_dict)
x_test['attack_cat'] = x_test['attack_cat'].replace(replace_dict)


In [12]:
# Convert numpy arrays back to DataFrames for saving
x_train_df = pd.DataFrame(x_train)
x_test_df = pd.DataFrame(x_test)

# Save to CSV files
x_train_df.to_csv('train15_xy.csv', index=False)
x_test_df.to_csv('test15_xy.csv', index=False)

